In [5]:
import streamlit as st
import numpy as np
import pandas as pd
import utils.web_util as wu
import os
import utils.utilities as ut
from pathlib import Path
from scipy.optimize import minimize

# BASE_DIR will be the directory where the current Jupyter Notebook file is located.
BASE_DIR = Path().resolve()

equity_data = pd.read_csv(BASE_DIR / "dataImporter/equity_data.csv")
commodity_data = pd.read_csv(BASE_DIR / "dataImporter/commodities_data.csv")
crypto_data = pd.read_csv(BASE_DIR / "dataImporter/cryptos_data.csv")
bonds_data = pd.read_csv(BASE_DIR / "dataImporter/bonds_data.csv")

# Create ESG data
equity_data_esg = ut.equity_to_esg(equity_data)
commodity_data_esg = ut.commodity_to_esg(commodity_data)

# Clean crypto to adjust trading days
crypto_data['Date'] = pd.to_datetime(crypto_data['Date'])
crypto_data = crypto_data.set_index('Date')
crypto_data = crypto_data.loc[crypto_data.index.isin(equity_data['Date'])]


# Compute daily returns
equity_returns = ut.dailyreturns(equity_data)
equity_esg_returns = ut.dailyreturns(equity_data_esg)
commodity_returns = ut.dailyreturns(commodity_data)
commodity_esg_returns = ut.dailyreturns(commodity_data_esg)
crypto_returns = ut.dailyreturns(crypto_data)
bonds_returns = ut.dailyreturns(bonds_data)

# Functions for Optimization
def portfolio_risk(weights, cov_matrix):
    """Calculate the portfolio risk (standard deviation)"""
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

def risk_contribution(weights, cov_matrix):
    """Calculate the risk contribution of each asset in the portfolio"""
    portfolio_std = portfolio_risk(weights, cov_matrix)
    # Marginal contribution to risk
    marginal_contribution = np.dot(cov_matrix, weights) / portfolio_std
    # Risk contribution
    risk_contrib = weights * marginal_contribution
    return risk_contrib

def objective_function(weights, cov_matrix):
    """Minimize function to equalize risk contributions"""
    risk_contribs = risk_contribution(weights, cov_matrix)
    # We want all risk contributions to be equal, so minimize the sum of squared differences
    # from the average risk contribution.
    return np.sum((risk_contribs - np.mean(risk_contribs))**2) * 1000

def compute_erc_weights(returns, output_file='erc_weights.csv'):
    """
    Compute ERC weights for each year and export to CSV
    """
    weights_data = []
    
    # Iterate through the years from 2016 to 2024
    for year in range(2016, 2025):
        print(f"Computing weights for year {year}...")
        
        # Filter data for the current year
        daily_returns_year = returns.loc[returns.index.year == year]
        
        # Drop columns with any NaN values in this year
        daily_returns_year = daily_returns_year.dropna(axis=1)
        
        # Store the column names for the current year
        columns = daily_returns_year.columns.tolist()
        
        # Calculate covariance matrix
        covariance_matrix_year = daily_returns_year.cov()
        
        # Calculate ERC weights for the current year
        num_assets_year = len(covariance_matrix_year.columns)
        initial_weights_year = np.array(num_assets_year * [1. / num_assets_year])
        constraints_year = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
        bounds_year = tuple((0, 1) for asset in range(num_assets_year))
        options = {'ftol': 1e-10, 'maxiter': 1000}
        
        result_year = minimize(objective_function, initial_weights_year, 
                              args=(covariance_matrix_year,),
                              method='SLSQP', bounds=bounds_year, 
                              constraints=constraints_year,
                              options=options, tol=1e-10)
        
        erc_weights_year = result_year.x
        
        # Store weights with asset names
        weight_dict = {'year': year}
        for asset, weight in zip(columns, erc_weights_year):
            weight_dict[asset] = weight
        
        weights_data.append(weight_dict)
    
    # Create DataFrame
    weights_df = pd.DataFrame(weights_data)
    
    # Export to CSV
    weights_df.to_csv(output_file, index=False)
    print(f"\nWeights exported to {output_file}")
    
    return weights_df

weights_eq = compute_erc_weights(equity_returns, output_file='erc_weights_equity.csv')
weights_eqesg = compute_erc_weights(equity_esg_returns, output_file='erc_weights_equity_esg.csv')
weights_com = compute_erc_weights(commodity_returns, output_file='erc_weights_commodity.csv')
weights_comseg = compute_erc_weights(commodity_esg_returns, output_file='erc_weights_commodity_esg.csv')
weights_crypto = compute_erc_weights(crypto_returns, output_file='erc_weights_crypto.csv')


C:\Users\Yann\AppData\Local\Temp\ipykernel_1536\2340683916.py:25: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  crypto_data = crypto_data.loc[crypto_data.index.isin(equity_data['Date'])]
C:\Users\Yann\Downloads\quantitative-risk-management---ERC-portfolio-main\streamlit\utils\utilities.py:16: FutureWarning: The 'fill_method' keyword being not None and the 'limit' keyword in DataFrame.pct_change are deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_returns = dailyprice.pct_change(fill_method='ffill')
C:\Users\Yann\Downloads\quantitative-risk-management---ERC-portfolio-main\streamlit\utils\utilities.py:16: FutureWarning: The 'fill_method' keyword being n

Computing weights for year 2016...
Computing weights for year 2017...
Computing weights for year 2018...
Computing weights for year 2019...
Computing weights for year 2020...
Computing weights for year 2021...
Computing weights for year 2022...
Computing weights for year 2023...
Computing weights for year 2024...

Weights exported to erc_weights_equity.csv
Computing weights for year 2016...
Computing weights for year 2017...
Computing weights for year 2018...
Computing weights for year 2019...
Computing weights for year 2020...
Computing weights for year 2021...
Computing weights for year 2022...
Computing weights for year 2023...
Computing weights for year 2024...

Weights exported to erc_weights_equity_esg.csv
Computing weights for year 2016...
Computing weights for year 2017...
Computing weights for year 2018...
Computing weights for year 2019...
Computing weights for year 2020...
Computing weights for year 2021...
Computing weights for year 2022...
Computing weights for year 2023..